In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/engineering-placements-prediction/collegePlace.csv


# Placement Prediction Using XGB Classifier and Randomized Search CV 

# Importing Necessary Libraries

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import seaborn as sns

# Reading Data

In [3]:
data = pd.read_csv('/kaggle/input/engineering-placements-prediction/collegePlace.csv')

In [4]:
data.head()

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot
0,22,Male,Electronics And Communication,1,8,1,1,1
1,21,Female,Computer Science,0,7,1,1,1
2,22,Female,Information Technology,1,6,0,0,1
3,21,Male,Information Technology,0,8,0,1,1
4,22,Male,Mechanical,0,8,1,0,1


# Preprocessing Data

## Checking for Null values

In [5]:
data.isnull().sum()

Age                  0
Gender               0
Stream               0
Internships          0
CGPA                 0
Hostel               0
HistoryOfBacklogs    0
PlacedOrNot          0
dtype: int64

## Checking for datatypes

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2966 entries, 0 to 2965
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Age                2966 non-null   int64 
 1   Gender             2966 non-null   object
 2   Stream             2966 non-null   object
 3   Internships        2966 non-null   int64 
 4   CGPA               2966 non-null   int64 
 5   Hostel             2966 non-null   int64 
 6   HistoryOfBacklogs  2966 non-null   int64 
 7   PlacedOrNot        2966 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 185.5+ KB


## Gender and Stream are object type

## Encoding Category Data

In [7]:
data['Gender'].unique()

array(['Male', 'Female'], dtype=object)

In [8]:
data['Stream'].unique()

array(['Electronics And Communication', 'Computer Science',
       'Information Technology', 'Mechanical', 'Electrical', 'Civil'],
      dtype=object)

In [9]:
en = LabelEncoder()
catCols = ['Gender','Stream']
for cols in catCols:
    data[cols] = en.fit_transform(data[cols])

In [10]:
data.head()

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot
0,22,1,3,1,8,1,1,1
1,21,0,1,0,7,1,1,1
2,22,0,4,1,6,0,0,1
3,21,1,4,0,8,0,1,1
4,22,1,5,0,8,1,0,1


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2966 entries, 0 to 2965
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Age                2966 non-null   int64
 1   Gender             2966 non-null   int64
 2   Stream             2966 non-null   int64
 3   Internships        2966 non-null   int64
 4   CGPA               2966 non-null   int64
 5   Hostel             2966 non-null   int64
 6   HistoryOfBacklogs  2966 non-null   int64
 7   PlacedOrNot        2966 non-null   int64
dtypes: int64(8)
memory usage: 185.5 KB


# Splitting Data Into X and Y

In [12]:
Y = data["PlacedOrNot"]
X = data.drop("PlacedOrNot",axis = 1)

In [13]:
Y=pd.DataFrame(Y)

In [14]:
X.head()

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs
0,22,1,3,1,8,1,1
1,21,0,1,0,7,1,1
2,22,0,4,1,6,0,0
3,21,1,4,0,8,0,1
4,22,1,5,0,8,1,0


In [15]:
Y.head()

,PlacedOrNot
0,1
1,1
2,1
3,1
4,1


# Splitting Test Train Data

In [16]:
# Break off validation set from training data
X_train_full, X_valid_full, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# XGB Classifier

In [17]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
# Define the model

    
    # defining the model
clf = XGBClassifier(learning_rate=0.09, 
                    n_estimators=100, 
                    use_label_encoder=False,
                    random_state=42)
    
clf.fit(X_train_full, Y_train, eval_metric='logloss')
predictions = clf.predict(X_valid_full) 
print("accuracy_score: " + str(accuracy_score(Y_valid, predictions)))

accuracy_score: 0.9023569023569024


In [18]:
print("f1_score: " + str(f1_score(Y_valid, predictions)))

f1_score: 0.9067524115755629


In [19]:
predictions


array([1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,

In [20]:
X.head(18)

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs
0,22,1,3,1,8,1,1
1,21,0,1,0,7,1,1
2,22,0,4,1,6,0,0
3,21,1,4,0,8,0,1
4,22,1,5,0,8,1,0
5,22,1,3,0,6,0,0
6,21,1,1,0,7,0,1
7,21,1,4,1,7,0,0
8,21,1,1,2,6,0,0
9,21,0,1,1,6,1,0


In [21]:
data['Stream'].unique()

array([3, 1, 4, 5, 2, 0])

# Using Randomized Search CV

In [22]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# lr
learning_rate = [ float(x) for x in [0.9, 0.1, 0.09, 0.01]]

In [23]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               }

print(random_grid)


{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200], 'learning_rate': [0.9, 0.1, 0.09, 0.01]}


In [24]:
clf2 = XGBClassifier()

In [25]:
model = RandomizedSearchCV(estimator = clf2, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [26]:
model.fit(X_train_full, Y_train, eval_metric='logloss')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=400, learning_rate=0.09 ............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] ............. n_estimators=400, learning_rate=0.09, total=   0.4s
[CV] n_estimators=400, learning_rate=0.09 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] ............. n_estimators=400, learning_rate=0.09, total=   0.4s
[CV] n_estimators=400, learning_rate=0.09 ............................
[CV] ............. n_estimators=400, learning_rate=0.09, total=   0.4s
[CV] n_estimators=400, learning_rate=0.09 ............................
[CV] ............. n_estimators=400, learning_rate=0.09, total=   0.4s
[CV] n_estimators=400, learning_rate=0.09 ............................
[CV] ............. n_estimators=400, learning_rate=0.09, total=   0.4s
[CV] n_estimators=500, learning_rate=0.01 ............................
[CV] ............. n_estimators=500, learning_rate=0.01, total=   0.5s
[CV] n_estimators=500, learning_rate=0.01 ............................
[CV] ............. n_estimators=500, learning_rate=0.01, total=   0.5s
[CV] n_estimators=500, learning_rate=0.01 ............................
[CV] ............. n_estimators=500, learning_rate=0.01, total=   0.5s
[CV] n_estimators=500, learning_rate=0.01 ............................
[CV] .

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   19.9s finished


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                          

In [27]:
model.best_params_

{'n_estimators': 100, 'learning_rate': 0.1}

In [28]:
predictions2 = model.predict(X_valid_full) 
print("accuracy_score: " + str(accuracy_score(Y_valid, predictions2)))

accuracy_score: 0.8956228956228957


# Making Predictions

## Uncomment the section for realtime input

In [29]:
"""print("Enter Age : ")
age=int(input())
print("\nEnter Gender(M/F) : ")
gen=input()
if(gen=='M'):
    gender=1
    gender=int(gender)
elif(gen=='F'):
    gender=0
    gender=int(gender)
print("\nEnter Stream(ECE/CS/IT/MEC/ELE/CIV) : ")
stre=input()
if(stre=='ECE'):
    stream=3
elif(stre=='CS'):
    stream=1
elif(stre=='IT'):
    stream=4
elif(stre=='MEC'):
    stream=5
elif(stre=='ELE'):
    stream=2
elif(stre=='CIV'):
    stream=0
stream=int(stream)
print("\nEnter Number of Internships Done : ")
intern = int(input())
print("\nEnter CGPA : ")
cgpa=int(input())
print("\nEnter Hostel Accommodation : ")
hostel=int(input())
print("\nEnter number of backlogs : ")
back=int(input())"""
age=20
gender=1
stream=1
intern=3
cgpa=8
hostel=1
back=1
pred = clf.predict(np.array([[age,gender,stream,intern,cgpa,hostel,back]]))
if(pred==1):
    print("\nYou have high chances of getting placed")
else:
    print("\nYou have high chances of getting placed. Work more Hard!")


You have high chances of getting placed
